In [4]:
#read the csv files
import pandas as pd
import pyModeS as pms
import csv
import glob
from datetime import datetime

csv_files = glob.glob("*.csv")

df_list = [pd.read_csv(file) for file in csv_files]

df_combined = pd.concat(df_list, ignore_index=True)
df_combined.to_csv("february_data.csv", index=False)

In [5]:
#function: time to seconds
def time_to_seconds(time_str):
    if isinstance(time_str, str) and len(time_str) == 8 and time_str[2] == ':' and time_str[5] == ':':
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    else:
        return None 

In [ ]:
#decode the ads b messages
aircraft_data = {}
lat_ref, lon_ref = 38.4627, -9.0803

df = pd.read_csv('february_data.csv', names=["Date", "Time", "Data", "Correlation"], low_memory=False)

with open('sdr_fev_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Time", "Time_Sec", "ICAO", "Callsign", "Latitude", "Longitude", "Altitude", "Speed", "Track Angle", "Vertical Rate"])

    for _, row in df.iterrows():
        message = row["Data"]

        if pms.df(message) == 17:
            icao = pms.adsb.icao(message)
            type_code = pms.adsb.typecode(message)

            if icao not in aircraft_data:
                aircraft_data[icao] = {
                    "Date": [],
                    "Time": [],
                    "Time_Sec": [],
                    "ICAO": icao,
                    "Callsign": None,
                    "Latitude": None,
                    "Longitude": None,
                    "Altitude": None,
                    "Speed": None,
                    "Track Angle": None,
                    "Vertical Rate": None,
                }
                
            time_sec = time_to_seconds(row["Time"])
            if time_sec is not None:
                aircraft_data[icao]["Time_Sec"].append(time_sec)

            aircraft_data[icao]["Time"].append(row["Time"])

            formatted_date = datetime.strptime(row["Date"], "%a %b %d %Y").strftime("%Y-%m-%d")
            aircraft_data[icao]["Date"].append(formatted_date)

            if 1 <= type_code <= 4:
                callsign = pms.adsb.callsign(message)
                aircraft_data[icao]["Callsign"] = callsign

            elif 9 <= type_code <= 18:
                position = pms.adsb.position_with_ref(message, lat_ref, lon_ref)
                altitude = pms.adsb.altitude(message)
                if position is not None:
                    aircraft_data[icao]["Latitude"] = position[0]
                    aircraft_data[icao]["Longitude"] = position[1]
                if altitude is not None:
                    aircraft_data[icao]["Altitude"] = altitude 

            elif type_code == 19:
                velocity_data = pms.adsb.velocity(message)
                
                if velocity_data is not None:
                    speed, track, vertical_rate, airspeed = velocity_data
                    aircraft_data[icao]["Speed"] = speed 
                    aircraft_data[icao]["Track Angle"] = track 
                    aircraft_data[icao]["Vertical Rate"] = vertical_rate 
                    
                else:
                    aircraft_data[icao]["Speed"] = None
                    aircraft_data[icao]["Track Angle"] = None
                    aircraft_data[icao]["Vertical Rate"] = None
                    
            writer.writerow([
                aircraft_data[icao]["Date"][-1],  
                aircraft_data[icao]["Time"][-1],  
                aircraft_data[icao]["Time_Sec"][-1],  
                aircraft_data[icao]["ICAO"],
                aircraft_data[icao]["Callsign"],
                aircraft_data[icao]["Latitude"],
                aircraft_data[icao]["Longitude"],
                aircraft_data[icao]["Altitude"],
                aircraft_data[icao]["Speed"],   
                aircraft_data[icao]["Track Angle"],
                aircraft_data[icao]["Vertical Rate"],  
            ])

In [ ]:
#take the _ from the callsign and maintain only the current month

sdr_df = pd.read_csv('sdr_fev_data.csv')
sdr_df["Callsign"] = sdr_df["Callsign"].str.strip().str.rstrip("_") 

sdr_df['Date'] = pd.to_datetime(sdr_df['Date'], errors='coerce')
sdr_df = sdr_df[sdr_df['Date'].dt.month == 2] 

sdr_df.to_csv("sdr_fev25_data.csv", index=False)